In [271]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [272]:
def load_data():
    data = pd.read_csv('historical_data.csv', index_col='Date')
    data.index = pd.to_datetime(data.index, utc=True, format='ISO8601')
    data.index = data.index.date

    symbol = data.symbol.unique().tolist()[0]
    df = data[data.symbol == symbol].copy()
    df.index = pd.to_datetime(df.index)
    df.shape

    df['buy'] = 0
    df['sell'] = 0
    df['shares'] = 0
    df['buy_in'] = 0.0
    df['current_value'] = 0.0
    df['investment'] = 0.0
    df['profit'] = 0.0
    df['total_profit'] = 0.0
    df['action'] = None
    df['action_code'] = 0
    return df

In [273]:
def buy(df, investment_size, i):
    df.loc[df.index[i], 'buy']           = investment_size	 
    df.loc[df.index[i], 'sell']          = 0	 
    df.loc[df.index[i], 'shares']        = df.loc[df.index[i-1], 'shares'] + investment_size
    df.loc[df.index[i], 'buy_in']        = (df.loc[df.index[i-1], 'investment'] + (df.loc[df.index[i], 'actual'] * investment_size)) / df.loc[df.index[i], 'shares']
    df.loc[df.index[i], 'current_value'] = df.loc[df.index[i], 'shares'] * df.loc[df.index[i], 'actual']
    df.loc[df.index[i], 'investment']    = df.loc[df.index[i-1], 'investment'] + ( df.loc[df.index[i], 'actual'] * investment_size )
    df.loc[df.index[i], 'profit']        = 0    
    df.loc[df.index[i], 'total_profit']  = df.loc[df.index[i-1], 'total_profit']
    return df

In [274]:
def sell(df, i):
    profit = df.loc[df.index[i-1], 'current_value'] - df.loc[df.index[i-1], 'investment']  
    df.loc[df.index[i], 'buy']           = 0
    df.loc[df.index[i], 'sell']          = df.loc[df.index[i-1], 'shares']
    df.loc[df.index[i], 'shares']        = 0
    df.loc[df.index[i], 'buy_in']        = 0
    df.loc[df.index[i], 'current_value'] = 0
    df.loc[df.index[i], 'investment']    = 0
    df.loc[df.index[i], 'profit']        = profit
    df.loc[df.index[i], 'total_profit']  = df.loc[df.index[i-1], 'total_profit'] + profit
    return df

In [275]:
def hold(df, i):
    df.loc[df.index[i], 'buy']           = 0
    df.loc[df.index[i], 'sell']          = 0
    df.loc[df.index[i], 'shares']        = df.loc[df.index[i-1], 'shares']
    df.loc[df.index[i], 'buy_in']        = df.loc[df.index[i-1], 'buy_in']
    df.loc[df.index[i], 'current_value'] = df.loc[df.index[i], 'shares'] * df.loc[df.index[i], 'actual']
    df.loc[df.index[i], 'investment']    = df.loc[df.index[i-1], 'investment']
    df.loc[df.index[i], 'profit']        = df.loc[df.index[i], 'current_value'] - df.loc[df.index[i], 'investment']
    df.loc[df.index[i], 'total_profit']  = df.loc[df.index[i-1], 'total_profit']
    return df

In [276]:
limit = 20000
investment_size = 20
time_window = pd.Timedelta(days=5)

In [283]:
total_profit = 0
for i in range(len(df) - 1):
    limit_reached = False
    current_actual = df.loc[df.index[i], 'actual']
    next_actual_values = df.loc[df.index[i + 1:i + 1 + time_window.days], 'actual']
    
    # Buy
    if current_actual < next_actual_values.min():
        limit_reached = df.loc[df.index[i-1], 'investment'] + (investment_size * df.loc[df.index[i-1], 'actual']) > limit           
        if not limit_reached:
            df = buy(df, investment_size, i)
            df.loc[df.index[i], 'action'] = 'buy'
        else:
            df.loc[df.index[i], 'action'] = 'limit'
            df = hold(df, i)            
            continue

    # Sell        
    elif current_actual > next_actual_values.max():        
        if df.loc[df.index[i-1], 'shares'] == 0:
            df = hold(df, i)
            df.loc[df.index[i], 'action'] = '(sell)'
            continue
        else:
            df = sell(df, i)          
            df.loc[df.index[i], 'action'] = 'sell'
    # Hold
    else:
        df.loc[df.index[i], 'action'] = 'hold'
        df.loc[df.index[i], 'action_code'] = 0
        df = hold(df, i)
total_profit = df.total_profit.iloc[-2].astype(int)
total_profit

19908

In [278]:
# remove the last row of the dataframe
df = df[:-1]

# add buy, limit and sell points to the dataframe
df['buy_point'] = df.apply(lambda x: x.actual if x.action == 'buy' else np.nan, axis=1)
df['limit_point'] = df.apply(lambda x: x.actual if x.action == 'limit' else np.nan, axis=1)
df['sell_point'] = df.apply(lambda x: x.actual if x.action == 'sell' else np.nan, axis=1)

total_profit = df.total_profit.iloc[-1].ast

9961

In [279]:
# Create subplots, one for each trace
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

# Add each trace to a separate subplot
fig.add_trace(go.Scatter(x=df.index, y=df.actual     , mode='lines'  , marker=dict(color='blue'  , size=5), name='actual'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.buy_point  , mode='markers', marker=dict(color='green' , size=5), name='buy'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.limit_point, mode='markers', marker=dict(color='yellow', size=2), name='sell'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.sell_point , mode='markers', marker=dict(color='red'   , size=2), name='sell'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df.profit, mode='lines', name='profit'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.shares, mode='lines', name='shares'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.investment, mode='lines', name='investment'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.total_profit, mode='lines', name='total_profit'), row=3, col=1)


# Update layout
fig.update_layout(title=f"[{symbol}] Total profit: {total_profit:,} €".replace(',', '.'), template='plotly_dark', height=800)

# Add titles to the subplots
fig.update_yaxes(title_text="Price and Points", row=1, col=1)
fig.update_yaxes(title_text="Total Profit", row=2, col=1)

# Show the figure
fig.show()
pd.concat([df.select_dtypes(include='number').fillna(0).astype(int), df[['action']]], axis=1)

,actual,buy,sell,shares,buy_in,current_value,investment,profit,total_profit,action_code,buy_point,limit_point,sell_point,action
2023-02-24,145,0,0,0,0,0,0,0,0,0,0,0,0,hold
2023-02-27,147,0,0,0,0,0,0,0,0,0,0,0,0,hold
2023-02-28,146,0,0,0,0,0,0,0,0,0,0,0,0,hold
2023-03-01,144,20,0,20,144,2890,2890,0,0,0,144,0,0,buy
2023-03-02,145,20,0,40,144,5805,5793,0,0,0,145,0,0,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-15,183,0,0,0,0,0,0,0,9946,0,0,0,0,hold
2024-02-16,182,0,0,0,0,0,0,0,9946,0,0,0,0,hold
2024-02-20,181,20,0,20,181,3631,3631,0,9946,0,181,0,0,buy
2024-02-21,182,20,0,40,181,7292,7277,0,9946,0,182,0,0,buy


In [280]:
df.head(50)

,actual,symbol,buy,sell,shares,buy_in,current_value,investment,profit,total_profit,action,action_code,buy_point,limit_point,sell_point
2023-02-24,145.931076,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,hold,0,NaN,NaN,NaN
2023-02-27,147.134644,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,hold,0,NaN,NaN,NaN
2023-02-28,146.627335,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,hold,0,NaN,NaN,NaN
2023-03-01,144.538498,AAPL,20,0,20,144.538498,2890.769958,2890.769958,0.000000,0.000000,buy,0,144.538498,NaN,NaN
2023-03-02,145.135330,AAPL,20,0,40,144.836914,5805.413208,5793.476562,0.000000,0.000000,buy,0,145.135330,NaN,NaN
2023-03-03,150.228134,AAPL,0,0,40,144.836914,6009.125366,5793.476562,215.648804,0.000000,hold,0,NaN,NaN,NaN
2023-03-06,153.013275,AAPL,0,40,0,0.000000,0.000000,0.000000,215.648804,215.648804,sell,0,NaN,NaN,153.013275
2023-03-07,150.795120,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,215.648804,hold,0,NaN,NaN,NaN
2023-03-08,152.058365,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,215.648804,hold,0,NaN,NaN,NaN
2023-03-09,149.790466,AAPL,0,0,0,0.000000,0.000000,0.000000,0.000000,215.648804,hold,0,NaN,NaN,NaN
